In [ ]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.server
import datetime
import pandas as pd
import pickle
import numpy

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

port = 8008
server = SimpleXMLRPCServer(("localhost", port), 
                            requestHandler=RequestHandler)

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname) 
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
server.register_function(black_list_check, 'black_list_check')

# Проверка на наличие человека в черном списке с проверкой по ФИО и дате рождения
def black_list_check_fio(sname, name, patronym, birth):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    #exist = any(frame['Surname'] == sname) and any(frame['Name'] == name) and any(frame['Patronym'] == patronym) and any(frame['Birth'] == birth)
    #if (exist):
    records = frame.query('Surname==\"'+sname+'\" & Name==\"'+name+'\" & Patronym==\"'+patronym+ '\" & Birth==\"'+birth+'\"')
    if (len(records)>0):
        return sname+ " "+name + " "+patronym +" ("+birth +"): bad_boy"
    else:
        return sname+ " "+name + " "+patronym +" ("+birth +"): good_boy"
server.register_function(black_list_check_fio, 'black_list_check_fio')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'back_binary')
    
# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
# или монохромные изображения размерностью (M, N, 1)
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1] 
    if (type(img_arr[0][0]) == numpy.uint8): 
        for i in range(height):
            for j in range(width): 
                img_arr[i][j] = 255 - img_arr[i][j]
    else:
        for i in range(height):
            for j in range(width):
                img_arr[i][j][0] = 255 - img_arr[i][j][0]
                img_arr[i][j][1] = 255 - img_arr[i][j][1]
                img_arr[i][j][2] = 255 - img_arr[i][j][2]
             
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)  
server.register_function(send_back_inversion, 'color_inversion')

# Разворот изображения относительно вертикали для цветных и монохромных изображений
def display_image_vertically(bin_data):
    img_arr = pickle.loads(bin_data.data)
    img_arr_copy = img_arr.copy()
    height = img_arr.shape[0]
    width = img_arr.shape[1] 
    if (type(img_arr[0][0]) == numpy.uint8):
        for i in range(height):
            for j in range(width): 
                img_arr_copy[i][j] = img_arr[i][width-1-j]  
    else: 
        for i in range(height):
            for j in range(width):
                img_arr_copy[i][j][0] =  img_arr[i][width-j-1][0] 
                img_arr_copy[i][j][1] =  img_arr[i][width-j-1][1] 
                img_arr_copy[i][j][2] =  img_arr[i][width-j-1][2] 
        
    pimg = pickle.dumps(img_arr_copy)        
    return xmlrpc.client.Binary(pimg) 
server.register_function(display_image_vertically, 'display_image_vertically')

# Бинаризация изображения по задаваемому порогу 1-255 для цветных и монохромных изображений
def binarize_image_by_threshold(bin_data, threshold):  
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1] 
    if (type(img_arr[0][0]) == numpy.uint8):
        for i in range(height):
            for j in range(width): 
                if (img_arr[i][j] < threshold): 
                    img_arr[i][j] = 0 
                else: 
                    img_arr[i][j] = 255   
    else:
        for i in range(height):
            for j in range(width):
                    if (img_arr[i][j][0] < threshold): 
                        img_arr[i][j][0] = 0 
                    else: 
                        img_arr[i][j][0] = 255
                    
                    if (img_arr[i][j][1] < threshold): 
                        img_arr[i][j][1] = 0 
                    else: 
                        img_arr[i][j][1] = 255
                    
                    if (img_arr[i][j][2] < threshold): 
                        img_arr[i][j][2] = 0 
                    else: 
                        img_arr[i][j][2] = 255
            
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(binarize_image_by_threshold, 'binarize_image_by_threshold')



print("Listening on port "+str(port)+"...")
server.serve_forever()


Listening on port 8008...


127.0.0.1 - - [24/Sep/2023 21:13:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:14] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:16] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:18] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:20] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:22] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:24] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:26] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:28] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:30] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:32] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:36] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/2023 21:13:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [24/Sep/202